<a href="https://colab.research.google.com/github/camimariaf/ProjetoP3ReservaHotel/blob/main/ProjetoSoftwareReservaHotel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import datetime
from typing import Dict, List, Optional
from enum import Enum
from dataclasses import dataclass
import uuid # identificadores unicos universais (aleatorio)
from abc import ABC, abstractmethod

class Language(Enum):
    PORTUGUESE = "pt"
    ENGLISH = "en"
    SPANISH = "es"

class PaymentMethod(Enum):
    CREDIT_CARD = "credit_card"
    PIX = "pix"
    BANK_TRANSFER = "bank_transfer"

class PaymentStatus(Enum):
    PENDING = "pending"
    COMPLETED = "completed"
    FAILED = "failed"

@dataclass
class Review:
    customer_id: str
    hotel_id: str
    rating: int
    comment: str
    date: datetime

@dataclass
class Room:
    room_number: str
    room_type: str
    base_price: float
    amenities: List[str]
    is_available: bool = True

class PaymentProcessor(ABC):
    @abstractmethod
    def process_payment(self, amount: float) -> bool:
        pass

class CreditCardProcessor(PaymentProcessor):
    def process_payment(self, amount: float) -> bool:
        print(f"Processing credit card payment: ${amount:.2f}")
        return True

class PixProcessor(PaymentProcessor):
    def process_payment(self, amount: float) -> bool:
        print(f"Processing PIX payment: ${amount:.2f}")
        return True

class Person(ABC):
    def __init__(self, name: str, email: str, phone: str):
        self.id = str(uuid.uuid4())
        self.name = name
        self.email = email
        self.phone = phone

    @abstractmethod
    def get_role(self) -> str:
        pass

class Customer(Person):
    def __init__(self, name: str, email: str, phone: str, language: Language):
        super().__init__(name, email, phone)
        self.language = language
        self.loyalty_points = 0
        self.bookings: List[str] = []
        self.preferences: Dict[str, str] = {}

    def add_loyalty_points(self, points: int):
        self.loyalty_points += points

    def update_preferences(self, preferences: Dict[str, str]):
        self.preferences.update(preferences)

    def get_role(self) -> str:
        return f"Customer - {self.name} ({self.phone})"

class Employee(Person):
    def __init__(self, name: str, email: str, phone: str, department: str):
        super().__init__(name, email, phone)
        self.department = department

    def get_role(self) -> str:
        return f"Employee ({self.department}) - {self.name} ({self.email})"

class HotelEntity(ABC):
    def __init__(self, name: str, location: str, description: str):
        self.id = str(uuid.uuid4())
        self.name = name
        self.location = location
        self.description = description

class Hotel(HotelEntity):
    def __init__(self, name: str, location: str, description: str):
        super().__init__(name, location, description)
        self.rooms: Dict[str, Room] = {}
        self.amenities: List[str] = []
        self.reviews: List[Review] = []
        self.photos: List[str] = []
        self.rating: float = 0.0

    def add_room(self, room: Room):
        self.rooms[room.room_number] = room

    def update_room_availability(self, room_number: str, is_available: bool):
        if room_number in self.rooms:
            self.rooms[room_number].is_available = is_available

    def add_review(self, review: Review):
        self.reviews.append(review)
        self._update_rating()

    def _update_rating(self):
        if self.reviews:
            self.rating = sum(review.rating for review in self.reviews) / len(self.reviews)

    def add_photo(self, photo_url: str):
        self.photos.append(photo_url)

class Booking:
    def __init__(self, customer_id: str, hotel_id: str, room_number: str,
                 check_in: datetime, check_out: datetime, total_price: float):
        self.id = str(uuid.uuid4())
        self.customer_id = customer_id
        self.hotel_id = hotel_id
        self.room_number = room_number
        self.check_in = check_in
        self.check_out = check_out
        self.total_price = total_price
        self.status = "confirmed"
        self.payment_status = PaymentStatus.PENDING

class BookingSystem:
    def __init__(self):
        self.customers: Dict[str, Customer] = {}
        self.hotels: Dict[str, Hotel] = {}
        self.bookings: Dict[str, Booking] = {}
        self.payment_processors: Dict[PaymentMethod, PaymentProcessor] = {
            PaymentMethod.CREDIT_CARD: CreditCardProcessor(),
            PaymentMethod.PIX: PixProcessor()
        }

    def register_customer(self, name: str, email: str, phone: str, language: Language) -> Customer:
        customer = Customer(name, email, phone, language)
        self.customers[customer.id] = customer
        return customer

    def add_hotel(self, name: str, location: str, description: str) -> Hotel:
        hotel = Hotel(name, location, description)
        self.hotels[hotel.id] = hotel
        return hotel

    def create_booking(self, customer_id: str, hotel_id: str, room_number: str,
                      check_in: datetime, check_out: datetime) -> Optional[Booking]:
        if customer_id not in self.customers or hotel_id not in self.hotels:
            return None

        hotel = self.hotels[hotel_id]
        if room_number not in hotel.rooms or not hotel.rooms[room_number].is_available:
            return None

        nights = (check_out - check_in).days
        total_price = hotel.rooms[room_number].base_price * nights

        booking = Booking(customer_id, hotel_id, room_number, check_in, check_out, total_price)
        self.bookings[booking.id] = booking
        hotel.update_room_availability(room_number, False)

        self.customers[customer_id].add_loyalty_points(int(total_price / 10))
        self.customers[customer_id].bookings.append(booking.id)
        return booking

    def process_payment(self, booking_id: str, payment_method: PaymentMethod) -> bool:
        if booking_id not in self.bookings:
            return False

        booking = self.bookings[booking_id]
        processor = self.payment_processors[payment_method]

        if processor.process_payment(booking.total_price):
            booking.payment_status = PaymentStatus.COMPLETED
            return True

        booking.payment_status = PaymentStatus.FAILED
        return False

    def add_review(self, customer_id: str, hotel_id: str, rating: int, comment: str):
        if customer_id in self.customers and hotel_id in self.hotels:
            review = Review(customer_id, hotel_id, rating, comment, datetime.now())
            self.hotels[hotel_id].add_review(review)

    def generate_report(self) -> Dict:
        total_bookings = len(self.bookings)
        total_revenue = sum(booking.total_price for booking in self.bookings.values()
                          if booking.payment_status == PaymentStatus.COMPLETED)
        hotel_occupancy = {
            hotel.id: {
                'name': hotel.name,
                'occupancy_rate': sum(1 for room in hotel.rooms.values()
                                    if not room.is_available) / len(hotel.rooms) if hotel.rooms else 0,
                'average_rating': hotel.rating
            }
            for hotel in self.hotels.values()
        }

        return {
            'total_bookings': total_bookings,
            'total_revenue': total_revenue,
            'hotel_occupancy': hotel_occupancy
        }

    def initialize_data(self):
        hotel1 = self.add_hotel("Hotel Praia Azul", "Maceió, AL", "Ótima vista para o mar.")
        hotel2 = self.add_hotel("Hotel Serra Verde", "Chapada Diamantina, BA", "Rodeado por natureza exuberante.")
        hotel3 = self.add_hotel("Hotel Beira Mar", "Maceió, AL", "Ótima vista para o mar.")
        hotel4 = self.add_hotel("Hotel Mangabeiras", "Maceió, AL", "Bem localizado, próximo ao centro.")
        hotel5 = self.add_hotel("Hotel Menina Bonita", "Recife, PE", "Próximo a Orla.")
        hotel6 = self.add_hotel("Hotel Arco-Íris", "Salvador, BA", "Próximo ao Pelourinho.")

        # Adicionando quartos ao Hotel 1
        room1 = Room("101", "Standard", 150.0, ["Wi-Fi", "TV", "Ar-condicionado"])
        room2 = Room("102", "Luxo", 270.0, ["Wi-Fi", "TV", "Ar-condicionado", "Banheira"])
        room3 = Room("103", "Standard", 150.0, ["Wi-Fi", "TV", "Ar-condicionado"])
        room4 = Room("104", "Luxo", 350.0, ["Wi-Fi", "TV", "Ar-condicionado", "Piscina"])
        room5 = Room("105", "Standard", 150.0, ["Wi-Fi", "TV", "Ar-condicionado"])
        room6 = Room("106", "Luxo", 250.0, ["Wi-Fi", "TV", "Ar-condicionado", "Jacuzzi"])
        self.hotels[hotel1.id].add_room(room1)
        self.hotels[hotel1.id].add_room(room2)
        self.hotels[hotel1.id].add_room(room3)
        self.hotels[hotel1.id].add_room(room4)
        self.hotels[hotel1.id].add_room(room5)
        self.hotels[hotel1.id].add_room(room6)

        # Adicionando quartos ao Hotel 2
        room7 = Room("201", "Econômico", 100.0, ["Wi-Fi", "Ventilador"])
        room8 = Room("202", "Premium", 300.0, ["Wi-Fi", "TV", "Ar-condicionado", "Jacuzzi"])
        room9 = Room("203", "Standard", 150.0, ["Wi-Fi", "TV", "Ar-condicionado"])
        room10 = Room("204", "Luxo", 250.0, ["Wi-Fi", "TV", "Ar-condicionado", "Banheira"])
        self.hotels[hotel2.id].add_room(room7)
        self.hotels[hotel2.id].add_room(room8)
        self.hotels[hotel2.id].add_room(room9)
        self.hotels[hotel2.id].add_room(room10)

        # Adicionando quartos ao Hotel 3
        room11 = Room("301", "Econômico", 100.0, ["Wi-Fi", "Ventilador"])
        room12 = Room("302", "Premium", 300.0, ["Wi-Fi", "TV", "Ar-condicionado", "Jacuzzi"])
        room13 = Room("303", "Standard", 150.0, ["Wi-Fi", "TV", "Ar-condicionado"])
        room14 = Room("304", "Luxo", 250.0, ["Wi-Fi", "TV", "Ar-condicionado", "Banheira"])
        self.hotels[hotel3.id].add_room(room11)
        self.hotels[hotel3.id].add_room(room12)
        self.hotels[hotel3.id].add_room(room13)
        self.hotels[hotel3.id].add_room(room14)

        # Adicionando quartos ao Hotel 4
        room15 = Room("401", "Econômico", 100.0, ["Wi-Fi", "Ventilador"])
        room16 = Room("402", "Premium", 300.0, ["Wi-Fi", "TV", "Ar-condicionado", "Jacuzzi"])
        room17 = Room("403", "Standard", 150.0, ["Wi-Fi", "TV", "Ar-condicionado"])
        self.hotels[hotel4.id].add_room(room15)
        self.hotels[hotel4.id].add_room(room16)
        self.hotels[hotel4.id].add_room(room17)

        # Adicionando quartos ao Hotel 5
        room18 = Room("501", "Econômico", 100.0, ["Wi-Fi", "Ventilador"])
        room19 = Room("502", "Premium", 300.0, ["Wi-Fi", "TV", "Ar-condicionado", "Jacuzzi"])
        self.hotels[hotel5.id].add_room(room18)
        self.hotels[hotel5.id].add_room(room19)

        # Adicionando quartos ao Hotel 6
        room20 = Room("601", "Econômico", 100.0, ["Wi-Fi", "Ventilador"])
        room21 = Room("602", "Premium", 300.0, ["Wi-Fi", "TV", "Ar-condicionado", "Jacuzzi"])
        room22 = Room("603", "Standard", 150.0, ["Wi-Fi", "TV", "Ar-condicionado"])
        room23 = Room("604", "Luxo", 250.0, ["Wi-Fi", "TV", "Ar-condicionado", "Banheira"])
        self.hotels[hotel6.id].add_room(room20)
        self.hotels[hotel6.id].add_room(room21)
        self.hotels[hotel6.id].add_room(room22)
        self.hotels[hotel6.id].add_room(room23)

        print("Dados iniciais carregados: 6 hotéis e 23 quartos adicionados automaticamente.\n")

def interactive_menu(booking_system):

    while True:
        print("\n===== Hotel Booking System =====")
        print("1. Cadastrar Cliente")
        print("2. Adicionar Hotel")
        print("3. Adicionar Quarto ao Hotel")
        print("4. Criar Reserva")
        print("5. Processar Pagamento")
        print("6. Adicionar Avaliação")
        print("7. Gerar Relatório")
        print("8. Listar Reservas de um Cliente")
        print("9. Listar Hotéis Disponíveis")
        print("10. Modificar Reserva")
        print("11. Cancelar Reserva")
        print("12. Sair")

        choice = input("Escolha uma opção: ")

        if choice == "1":
            name = input("Nome: ")
            email = input("E-mail: ")
            phone = input("Telefone: ")
            language = input("Idioma (pt/en/es): ").lower()
            language = Language.PORTUGUESE if language == "pt" else Language.ENGLISH if language == "en" else Language.SPANISH
            customer = booking_system.register_customer(name, email, phone, language)
            print(f"Cliente cadastrado com ID: {customer.id}")

        elif choice == "2":
            name = input("Nome do Hotel: ")
            location = input("Localização: ")
            description = input("Descrição: ")
            hotel = booking_system.add_hotel(name, location, description)
            print(f"Hotel adicionado com ID: {hotel.id}")

        elif choice == "3":
            hotel_id = input("ID do Hotel: ")
            if hotel_id in booking_system.hotels:
                room_number = input("Número do Quarto: ")
                room_type = input("Tipo do Quarto: ")
                base_price = float(input("Preço Base: "))
                amenities = input("Amenidades (separadas por vírgula): ").split(",")
                room = Room(room_number, room_type, base_price, amenities)
                booking_system.hotels[hotel_id].add_room(room)
                print(f"Quarto {room_number} adicionado ao hotel {hotel_id}")
            else:
                print("Hotel não encontrado.")

        elif choice == "4":
            customer_id = input("ID do Cliente: ")
            hotel_id = input("ID do Hotel: ")
            room_number = input("Número do Quarto: ")
            check_in = datetime.strptime(input("Check-in (YYYY-MM-DD): "), "%Y-%m-%d")
            check_out = datetime.strptime(input("Check-out (YYYY-MM-DD): "), "%Y-%m-%d")
            booking = booking_system.create_booking(customer_id, hotel_id, room_number, check_in, check_out)
            if booking:
                print(f"Reserva criada com ID: {booking.id}")
            else:
                print("Erro ao criar reserva. Verifique os dados.")

        elif choice == "5":
            booking_id = input("ID da Reserva: ")
            payment_method = input("Método de Pagamento (credit_card/pix/bank_transfer): ")
            payment_method = PaymentMethod[payment_method.upper()]
            if booking_system.process_payment(booking_id, payment_method):
                print("Pagamento realizado com sucesso!")
            else:
                print("Falha no pagamento.")

        elif choice == "6":
            customer_id = input("ID do Cliente: ")
            hotel_id = input("ID do Hotel: ")
            rating = int(input("Nota (1-5): "))
            comment = input("Comentário: ")
            booking_system.add_review(customer_id, hotel_id, rating, comment)
            print("Avaliação adicionada!")

        elif choice == "7":
            report = booking_system.generate_report()
            print("\n===== Relatório do Sistema =====")
            print(f"Total de Reservas: {report['total_bookings']}")
            print(f"Receita Total: ${report['total_revenue']:.2f}")
            print("Ocupação dos Hotéis:")
            for hotel_id, data in report['hotel_occupancy'].items():
                print(f"{data['name']} - Ocupação: {data['occupancy_rate']*100:.2f}% - Avaliação Média: {data['average_rating']:.2f}")

        elif choice == "8":
          customer_id = input("Digite o ID do Cliente para ver suas reservas: ")
          if customer_id in booking_system.customers:
            customer = booking_system.customers[customer_id]
            if customer.bookings:
              print("\n============ Reservas do Cliente ============")
              for booking_id in customer.bookings:
                booking = booking_system.bookings[booking_id]
                print(f"ID da Reserva: {booking.id}, Hotel: {booking.hotel_id}, Quarto: {booking.room_number}")
                print(f"Check-in: {booking.check_in}, Check-out: {booking.check_out}, Preço: R${booking.total_price:.2f}")
            else:
              print("Nenhuma reserva encontrada para este cliente.")
          else:
            print("Cliente não encontrado.")

        elif choice == "9":
          print("\n============ Hotéis Disponíveis ============")
          if booking_system.hotels:
            for hotel_id, hotel in booking_system.hotels.items():
              print(f"ID: {hotel_id} - Nome: {hotel.name} - Localização: {hotel.location}")
              print(f"Descrição: {hotel.description}")
              print(f"Avaliação média: {hotel.rating:.2f}\n")
            hotel_id = input("Digite o ID do hotel para ver os quartos disponíveis: ")
            if hotel_id in booking_system.hotels:
              print("\n============ Quartos Disponíveis ============")
              for room_number, room in booking_system.hotels[hotel_id].rooms.items():
                if room.is_available:
                    print(f"Quarto {room_number} - Tipo: {room.room_type} - Preço: R${room.base_price:.2f}")
            else:
              print("Hotel não encontrado.")
          else:
            print("Nenhum hotel disponível no sistema.")

        elif choice == "10":
          booking_id = input("Digite o ID da reserva que deseja modificar: ")
          if booking_id in booking_system.bookings:
            booking = booking_system.bookings[booking_id]
            print(f"Reserva encontrada: Hotel {booking.hotel_id}, Quarto {booking.room_number}, Check-in {booking.check_in}, Check-out {booking.check_out}")

            new_check_in = datetime.strptime(input("Novo Check-in (YYYY-MM-DD): "), "%Y-%m-%d")
            new_check_out = datetime.strptime(input("Novo Check-out (YYYY-MM-DD): "), "%Y-%m-%d")

            nights = (new_check_out - new_check_in).days
            new_price = nights * booking_system.hotels[booking.hotel_id].rooms[booking.room_number].base_price
            booking.check_in = new_check_in
            booking.check_out = new_check_out
            booking.total_price = new_price

            print(f"Reserva modificada! Novo check-in: {booking.check_in}, Novo check-out: {booking.check_out}, Novo preço: R${new_price:.2f}")
          else:
            print("Reserva não encontrada.")


        elif choice == "11":
          booking_id = input("Digite o ID da reserva que deseja cancelar: ")
          if booking_id in booking_system.bookings:
            booking = booking_system.bookings[booking_id]
            hotel = booking_system.hotels[booking.hotel_id]

            hotel.update_room_availability(booking.room_number, True)  # Torna o quarto disponível novamente
            del booking_system.bookings[booking_id]  # Remove a reserva do sistema

            print("Reserva cancelada com sucesso!")
          else:
            print("Reserva não encontrada.")

        elif choice == "12":
            print("Saindo do sistema...")
            break

        else:
            print("Opção inválida. Tente novamente.")


def main():
    booking_system = BookingSystem()
    booking_system.initialize_data()
    interactive_menu(booking_system)

if __name__ == "__main__":
    main()


Dados iniciais carregados: 6 hotéis e 23 quartos adicionados automaticamente.


===== Hotel Booking System =====
1. Cadastrar Cliente
2. Adicionar Hotel
3. Adicionar Quarto ao Hotel
4. Criar Reserva
5. Processar Pagamento
6. Adicionar Avaliação
7. Gerar Relatório
8. Listar Reservas de um Cliente
9. Listar Hotéis Disponíveis
10. Modificar Reserva
11. Cancelar Reserva
12. Sair


KeyboardInterrupt: Interrupted by user